In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Data Preprocessing

pathFull = "/Users/Avaneesh/Library/Mobile Documents/com~apple~CloudDocs/Georgia Tech/AI:ML/COVID-19_Radiography_Dataset/"
# Set base path to dataset

pathCOV = pathFull + "COVID/images"
pathNorm = pathFull + "Normal/images"
cntCOVimg = len(os.listdir(pathCOV))
cntNormimg = len(os.listdir(pathNorm))
# Get image directories and count images for each class

COVimg = []
COVlab = []
COVname = []
for x in range(1,cntCOVimg + 1):
    imgName1 = "COVID-" + str(x) + ".png"
    tempPath1 = pathCOV + "/" + imgName1
    img1 = cv2.imread(tempPath1)
    img1 = np.float32(cv2.resize(img1, (100,100))) / 255.0
    COVimg.append(img1)
    COVlab.append(1)
    COVname.append(imgName1)
# Load, resize, and normalize COVID images; assign label 1

df = pd.DataFrame({"Image" : COVimg,"Label" : COVlab, "Image Name" : COVname})
# Create DataFrame for COVID images

Normimg = []
Normlab = []
Normname = []

rndm = list(range(1,cntNormimg+1))
random.shuffle(rndm)
newidx = rndm[:cntCOVimg]
for x in newidx:
    imgName2 = "Normal-" + str(x) + ".png"
    tempPath2 = pathNorm + "/" + imgName2
    img2 = cv2.imread(tempPath2)
    img2 = np.float32(cv2.resize(img2, (100,100))) / 255.0
    Normimg.append(img2)
    Normlab.append(0)
    Normname.append(imgName2)
# Randomly select and process Normal images to balance dataset; assign label 0

df2 = pd.DataFrame({"Image" : Normimg,"Label" : Normlab, "Image Name" : Normname})
# Create DataFrame for Normal images

data = pd.concat([df, df2], axis=0,ignore_index=False)
data = data.sample(frac=1,replace=True, ignore_index=True)
# Combine and shuffle both classes

x_train, x_test, y_train, y_test = train_test_split(data["Image"],data["Label"], test_size=.25)
# Split data into training and testing sets

In [ ]:
# Model Creation
COVmodel = Sequential([
    Input(shape=(100,100,3)),  # Input layer for 100x100 RGB images
    Conv2D(32, 3, activation='relu'),  # First convolutional layer
    MaxPooling2D(),                   # First pooling layer
    Conv2D(16, 3, activation='relu'), # Second convolutional layer
    MaxPooling2D(),                   # Second pooling layer
    Conv2D(16, 3, activation='relu'), # Third convolutional layer
    MaxPooling2D(),                   # Third pooling layer
    Flatten(),                        # Flatten feature maps
    Dense(512, activation='relu'),    # Dense layer with 512 units
    Dropout(0.5),                     # Dropout for regularization
    Dense(256, activation='relu'),    # Dense layer with 256 units
    Dropout(0.5),                     # Dropout for regularization
    Dense(1, activation='sigmoid')    # Output layer for binary classification
])

COVmodel.summary()


In [ ]:
# Model Compilation and Training
COVmodel.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
# Compile model with Adam optimizer and binary crossentropy loss

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
# Set up early stopping to prevent overfitting

x_train = np.array([img for img in x_train])
x_test = np.array([img for img in x_test])

y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)
# Convert data to numpy arrays

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
# Print shapes for verification

COVmodel.fit(x_train, y_train,epochs=25,validation_data=(x_test, y_test), callbacks=early_stopping)
# Train the model with early stopping and validation

In [ ]:
# Model Evaluation

plt.plot(COVmodel.history.history['accuracy'], label = 'train accuracy')
plt.plot(COVmodel.history.history['val_accuracy'],label = 'test accuracy')
plt.xlabel("Epoch #")
plt.ylabel("Percent Accuracy")
plt.legend()
plt.show()
# Plot training and validation accuracy over epochs

plt.plot(COVmodel.history.history['loss'], label = 'train loss')
plt.plot(COVmodel.history.history['val_loss'],label = 'test_loss')
plt.xlabel("Epoch #")
plt.ylabel("Percent Loss")
plt.legend()
plt.show()
# Plot training and validation loss over epochs

y_pred = (COVmodel.predict(x_test) > 0.5).astype(int).flatten()
# Predict on test set and threshold to get binary labels

cm = confusion_matrix(y_test, y_pred)
# Compute confusion matrix

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal", "COVID"])
disp.plot(cmap=plt.cm.Blues)
plt.show()
# Display confusion matrix as a plot

In [ ]:
# Grad-CAM Visualization

def get_img_array(img):
    # Ensure image is float32 and shape (1, 100, 100, 3)
    arr = np.expand_dims(img, axis=0)
    return arr

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Create a model that maps the input image to the activations of the last conv layer
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    # Compute the gradient of the top predicted class for the input image
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    # Get gradients of the class output value with respect to the feature map
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def plot_gradcam_for_img(img, model, last_conv_layer_name):
    
    # Prepare image for model
    img_array = get_img_array(img)
    # Generate Grad-CAM heatmap
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    # Prepare images for display
    img_disp = np.uint8(255 * img)
    heatmap_resized = cv2.resize(heatmap, (img_disp.shape[1], img_disp.shape[0]))
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img_disp, 0.6, heatmap_colored, 0.4, 0)
    # Plot results
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    plt.title("Original X-ray")
    plt.imshow(img)
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.title("Grad-CAM Heatmap")
    plt.imshow(heatmap, cmap='jet')
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.title("Superimposed")
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.tight_layout()
    plt.show()